# Formating the data
The goal for this notebook is to format the data into something that is useable to clean up the data.

### Imports
- tweepy - The data object that the data is saved in
- pickle - To unpickle the variables that have been saved
- pandas - To reprosent the data in a dataframe
- numpy - Working with arrays
- glob - Easy way to gather all the files in a folder

In [1]:
import tweepy
import pickle
import pandas
import numpy as np
import glob

## Testing the data

This is just to understand that data in each file and how to index it.
to get the information that we want we need to index the key firstly, then there will be a list of tweet data objects. To get the text of the tweet its then just ```.text```

The data comes in as such:
```{"user" : [{tweet}, {tweet}, {tweet}]}```

In [ ]:
# file = "12.p"

In [ ]:
# with open (file, "rb") as f:
#    data = pickle.load(f)
#    f.close()

## Collecting the data

This chunk of code collects all the names of the files that need to be opened so they can be user with the ```open()``` function to read all of the data in each file. 

The next chunk of code is similar to above where it collects all of the informaiont of the users tweets and adds them to a dictionary with an array of users and an array of users 1st, 2nd, ..., 1000th tweet. This seemed to be the easiest way to format the data to work with pandas data frame and show the data cohesivley. The data also needs to have the same amount of tweets for each user so we have to check if they have enough tweets before we start putting thier informaion in the data.

The last part of the code trys to append the tweet text to the tweet array, however, if it is the first tweet to go in then it has to create that array so the exception does so.

Finally we can add this dictionary to the data frame and show it in a nice looking table.

In [ ]:
files = []
for name in glob.glob('../tweets/*'):
    print (name)
    files.append(name)

In [ ]:
# userData = {
#     "User": np.array([]),
#     "tweets": np.array([])
# }
# for j in range (50):
#     file = files[j]
#     with open(file, 'rb') as f:
#         accounts = pickle.load(f)
#         for user, value in accounts.items():
#             combTweet = ""
#             if (len(value) < 1000):
#                 break
#             userData["User"]=np.append(userData["User"], [user])
#             for i in range(1000):
#                 tweet = value[i]
#                 combTweet = combTweet+ tweet.text
                
#             userData["tweets"] = np.append(userData['tweets'], [combTweet])        

# Making it more efficient
To make this more efficient on memory this code is going to have to be executed in batches and saved out to a file. I can do it a format like CSV or in jason form. For this task I think i'll use CSV.

There is an error at the moment where the last batch will crash as the index goes out of bounds of the array of files. This doesn't break everything just stops at the last batch.

In [ ]:
def saveToFile(data,file):
    with open (file, "a+", encoding="utf-8") as f:
        for i in range(len(data["User"])):
            key = data["User"][i]
            value = data["tweets"][i]
            f.write(f">{key},{value}\n")
        f.close()

In [ ]:
iter = 0
batchSize = 5
while True:
    userData = {
        "User": np.array([]),
        "tweets": np.array([])
    }
    for j in range (batchSize):
        if (j + (iter * batchSize) > len(files)):
            break
        file = files[j + (iter * batchSize)]
        with open(file, 'rb') as f:
            accounts = pickle.load(f)
            for user, tweets in accounts.items():
                combTweet = ""
                # does that user have more than 1000 tweets, if not move on to the next user
                if (len(tweets) < 1000):
                    break
                # add the user to the dictionary
                userData["User"]=np.append(userData["User"], [user])
                # gather all of the tweets and add them to the dictionary
                for i in range(1000):
                    # is the tweet a retweet or not
                    if (hasattr(tweets[i], "retweeted_status")):
                        break
                    tweet = tweets[i]
                    # Get the full text of the tweet without truncation
                    combTweet = combTweet + tweet.full_text.strip().replace('\n', '').replace(',', ' ')
                userData["tweets"] = np.append(userData['tweets'], [combTweet])
    # Increase the iterations to load more files
    iter += 1
    # Save out to a file
    saveToFile(userData, "userData.csv")

In [ ]:
df = pandas.DataFrame(userData)
df.style

In [ ]:
with open ('pickle/rawDataFrame.p', 'wb') as f:
    pickle.dump(df,f)
    f.close()